In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, f1_score, recall_score, precision_score, matthews_corrcoef, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm
from joblib import Parallel, delayed
import itertools
import seaborn as sns

# Data preprocessing function
def preprocess_data(data_path, group_mapping):
    try:
        data = pd.read_excel(data_path)
        data['Group'] = data['Group'].map(group_mapping)
        X = data.drop('Group', axis=1)
        y = data['Group']
        X.columns = X.columns.astype(str)
        
        # Apply MinMaxScaler
        scaler = MinMaxScaler()
        X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
        
        return X_scaled, y
    except Exception as e:
        print(f"Error in preprocess_data: {e}")
        return None, None

# Performance metrics calculation function
def calculate_performance_metrics(y_true, y_pred):
    try:
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp)
        mcc = matthews_corrcoef(y_true, y_pred)
        accuracy = accuracy_score(y_true, y_pred)
        
        return precision, recall, f1, specificity, mcc, cm, accuracy
    except Exception as e:
        print(f"Error in calculate_performance_metrics: {e}")
        return None, None, None, None, None, None, None

# Calculate the best threshold
def calculate_best_threshold(fpr, tpr, thresholds):
    try:
        J = tpr - fpr
        ix = np.argmax(J)
        best_threshold = thresholds[ix]
        
        return best_threshold, tpr[ix], fpr[ix], ix
    except Exception as e:
        print(f"Error in calculate_best_threshold: {e}")
        return None, None, None, None

# OOB accuracy and 95% confidence interval calculation
def calculate_oob_ci(oob_score, n):
    try:
        oob_se = np.sqrt(oob_score * (1 - oob_score) / n)
        z = norm.ppf(0.975)  # 1.96 for 95% CI
        ci_lower = oob_score - z * oob_se
        ci_upper = oob_score + z * oob_se
        
        return ci_lower, ci_upper
    except Exception as e:
        print(f"Error in calculate_oob_ci: {e}")
        return None, None

# Main function to analyze the data and plot the results
def analyze_data(data_path, label, best_params):
    # Load and preprocess data
    X_train, y_train = preprocess_data(data_path, {'NT': 0, 'T': 1})
    if X_train is None or y_train is None:
        print(f"Failed to preprocess data for {label}")
        return None, None

    # Train model and get feature importance
    rf_model = RandomForestClassifier(**best_params)
    rf_model.fit(X_train, y_train)
    feature_importances = rf_model.feature_importances_
    features = X_train.columns
    sorted_features = [feature for _, feature in sorted(zip(feature_importances, features), reverse=True)]
    top_5_feature_names = sorted_features[:5]
    print(f'{label} top_5_feature_names:')
    print(top_5_feature_names)

    # Calculate performance metrics using original data
    oob_probabilities = rf_model.oob_decision_function_[:, 1]
    fpr, tpr, thresholds = roc_curve(y_train, oob_probabilities)
    auc_score = roc_auc_score(y_train, oob_probabilities)

    # Calculate the best threshold
    best_threshold, best_tpr, best_fpr, ix = calculate_best_threshold(fpr, tpr, thresholds)
    if best_threshold is None:
        print(f"Failed to calculate best threshold for {label}")
        return None, None

    print(f'{label} Original cut-off value: {best_threshold}')
    print(f'{label} Original TPR: {best_tpr}, FPR: {best_fpr}')

    # Performance metrics
    oob_predictions = (oob_probabilities > 0.5).astype(int)
    precision, recall, f1, specificity, mcc, cm, accuracy = calculate_performance_metrics(y_train, oob_predictions)
    if precision is None:
        print(f"Failed to calculate performance metrics for {label}")
        return None, None

    # OOB accuracy and 95% confidence interval
    oob_accuracy = rf_model.oob_score_
    ci_lower, ci_upper = calculate_oob_ci(oob_accuracy, len(y_train))
    if ci_lower is None:
        print(f"Failed to calculate OOB CI for {label}")
        return None, None

    print(f"{label} Original OOB Accuracy: {oob_accuracy:.4f}")
    print(f"{label} Original 95% CI for Accuracy: ({ci_lower:.4f}, {ci_upper:.4f})")
    print(f"{label} Original Precision: {precision:.4f}, Recall (Sensitivity): {recall:.4f}, F1 Score: {f1:.4f}, Specificity: {specificity:.4f}, Accuracy: {accuracy:.4f}, MCC: {mcc:.4f}")

    metrics = ['Precision', 'Recall', 'F1 Score', 'Specificity', 'Accuracy', 'MCC']
    values_train = [precision, recall, f1, specificity, accuracy, mcc]

    original_results = (fpr, tpr, auc_score, values_train, metrics, label, best_threshold, ix, oob_accuracy, ci_lower, ci_upper, cm)

    # Function to evaluate model using OOB validation
    def evaluate_model_oob(feature_subset):
        X_train_selected = X_train[list(feature_subset)]
        model = RandomForestClassifier(**best_params)
        model.fit(X_train_selected, y_train)
        oob_probabilities = model.oob_decision_function_[:, 1]
        oob_auc = roc_auc_score(y_train, oob_probabilities)
        return oob_auc, feature_subset

    # Generate feature combinations using itertools
    all_feature_combinations = []
    for i in range(1, len(top_5_feature_names) + 1):
        combinations = list(itertools.combinations(top_5_feature_names, i))
        all_feature_combinations.extend(combinations)

    # Perform parallel computation using Parallel and delayed
    results = Parallel(n_jobs=-1)(delayed(evaluate_model_oob)(combo) for combo in all_feature_combinations)

    # Find the best combination
    best_result = max(results, key=lambda x: x[0])
    best_auc, best_combination = best_result

    print(f'{label} Best AUC: {best_auc:.4f}')
    print(f'{label} Best feature combination:', best_combination)

    # Plot ROC curve for the best combination
    X_train_best = X_train[list(best_combination)]
    model_best = RandomForestClassifier(**best_params)
    model_best.fit(X_train_best, y_train)
    oob_probabilities_best = model_best.oob_decision_function_[:, 1]
    fpr_best, tpr_best, thresholds_best = roc_curve(y_train, oob_probabilities_best)
    auc_score_best = roc_auc_score(y_train, oob_probabilities_best)

    # Calculate the best threshold for the best combination
    best_threshold_best, best_tpr_best, best_fpr_best, ix_best = calculate_best_threshold(fpr_best, tpr_best, thresholds_best)
    if best_threshold_best is None:
        print(f"Failed to calculate best threshold for {label} Best")
        return original_results, None

    print(f'{label} Best cut-off value: {best_threshold_best}')
    print(f'{label} Best TPR: {best_tpr_best}, FPR: {best_fpr_best}')

    best_results = (fpr_best, tpr_best, auc_score_best, best_threshold_best, ix_best)

    return original_results, best_results

# Define best parameter combinations
best_params_entire_dataset = {
    'bootstrap': True,
    'criterion': 'gini',
    'max_depth': 4,
    'max_features': None,
    'min_samples_leaf': 4,
    'min_samples_split': 2,
    'n_estimators': 500,
    'random_state': 42, 
    'oob_score': True
}

best_params_primary_lc = {
    'bootstrap': True,
    'criterion': 'entropy',
    'max_depth': 4,
    'max_features': 'sqrt',
    'min_samples_leaf': 4,
    'min_samples_split': 10, 
    'n_estimators': 500,
    'random_state': 42,
    'oob_score': True
}

# Analyze two datasets
data_path1 = 'Primary LC dataset.xlsx'
data_path2 = 'Entire dataset.xlsx'

results_primary, best_results_primary = analyze_data(data_path1, 'Primary LC', best_params_primary_lc)
results_entire, best_results_entire = analyze_data(data_path2, 'Entire Dataset', best_params_entire_dataset)

# Plot Confusion Matrix for Primary LC
if results_primary:
    fpr_primary, tpr_primary, auc_primary, values_primary, metrics_primary, label_primary, best_threshold_primary, ix_primary, oob_accuracy_primary, ci_lower_primary, ci_upper_primary, cm_primary = results_primary
    
    plt.figure(figsize=(8, 8), dpi=300)
    sns.heatmap(cm_primary, annot=True, fmt='d', cmap='Oranges', xticklabels=['Non-Tumor', 'Tumor'], yticklabels=['Non-Tumor', 'Tumor'], cbar=True)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix for Primary LC')
    plt.show()

# Plot Confusion Matrix for Entire Dataset
if results_entire:
    fpr_entire, tpr_entire, auc_entire, values_entire, metrics_entire, label_entire, best_threshold_entire, ix_entire, oob_accuracy_entire, ci_lower_entire, ci_upper_entire, cm_entire = results_entire
    
    plt.figure(figsize=(8, 8), dpi=300)
    sns.heatmap(cm_entire, annot=True, fmt='d', cmap='Greens', xticklabels=['Non-Tumor', 'Tumor'], yticklabels=['Non-Tumor', 'Tumor'], cbar=True)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix for Entire Dataset')
    plt.show()

# Plot ROC curve and model performance for Primary LC
if results_primary and best_results_primary:
    plt.figure(figsize=(8, 8), dpi=300)
    plt.plot(fpr_primary, tpr_primary, color='orange', lw=2, label=f'Primary LC (AUC = {auc_primary:.2f})')
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.scatter(fpr_primary[ix_primary], tpr_primary[ix_primary], marker='o', color='red', label=f'Primary LC Cut-off value: {best_threshold_primary:.2f}')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Primary LC ROC Curve', fontsize=16, fontweight='bold')
    plt.legend(loc='lower right', fontsize=14)
    plt.show()

    fig, ax = plt.subplots(figsize=(8, 8), dpi=300)
    x = range(len(metrics_primary))

    ax.plot(x, values_primary, marker='o', linestyle='-', color='orange', label='Primary LC')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_primary, fontsize=14)
    ax.set_ylabel('Score', fontsize=14)
    ax.set_title('Primary LC Model Performance', fontsize=16, fontweight='bold')
    ax.set_ylim([0, 1])
    ax.legend(loc='lower right', fontsize=14)

    plt.tight_layout()
    plt.show()

# Plot combined ROC curve
if results_primary and results_entire:
    plt.figure(figsize=(8, 8), dpi=300)
    plt.plot(fpr_primary, tpr_primary, color='orange', lw=2, label=f'Primary LC (AUC = {auc_primary:.2f})')
    plt.plot(fpr_entire, tpr_entire, color='green', lw=2, linestyle='-.', label=f'Entire Dataset (AUC = {auc_entire:.2f})')
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.scatter(fpr_primary[ix_primary], tpr_primary[ix_primary], marker='o', color='red', label=f'Primary LC Cut-off value: {best_threshold_primary:.2f}')
    plt.scatter(fpr_entire[ix_entire], tpr_entire[ix_entire], marker='x', color='red', label=f'Entire Dataset Cut-off value: {best_threshold_entire:.2f}')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Combined ROC Curve', fontsize=16, fontweight='bold')
    plt.legend(loc='lower right', fontsize=14)
    plt.show()

# Plot comparison of model performance for Primary LC and Entire Dataset
if results_primary and results_entire:
    fig, ax = plt.subplots(figsize=(8, 8), dpi=300)
    x = range(len(metrics_primary))

    ax.plot(x, values_primary, marker='o', linestyle='-', color='orange', label='Primary LC')
    ax.plot(x, values_entire, marker='x', linestyle='-.', color='green', label='Entire Dataset')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_primary, fontsize=14)
    ax.set_ylabel('Score', fontsize=14)
    ax.set_title('Primary LC vs Entire Dataset Model Performance', fontsize=16, fontweight='bold')
    ax.set_ylim([0, 1])
    ax.legend(loc='lower right', fontsize=14)

    plt.tight_layout()
    plt.show()
